In [2]:
import re
 
# reading given tsv file
with open("Datasets/14_graph.tsv", 'r') as myfile: 
  with open("Datasets/14_graph.csv", 'w') as csv_file:
    for line in myfile:
       
      # Replace every tab with comma
      fileContent = re.sub("\t", ",", line)
       
      # Writing into csv file
      csv_file.write(fileContent)
 
# output
print("Successfully made csv file")

Successfully made csv file


In [38]:
url1 = 'http://www.wikidata.org/entity/Q897357'
url2 = 'http://www.wikidata.org/prop/direct/P5021'
url3 = 'https://www.wikidata.org/wiki/Property:P5021'

In [110]:


from urllib.request import urlopen
import json

response = urlopen(url1)
data_json = json.loads(response.read())

In [ ]:
print(json.dumps(data_json, indent=4))

In [115]:
data_json['entities']['Q897357']['labels']['en']['value']

'The Ketchup Effect'

In [ ]:
import requests

response = requests.get(url2)
response.raise_for_status()
response.content

In [19]:
response.read()

b''

In [49]:
import pandas as pd

df = pd.read_csv('Datasets/14_graph.tsv', sep='\t', header=None)



In [50]:
df.columns

Int64Index([0, 1, 2], dtype='int64')

In [58]:
len(df.loc[df.iloc[:, 0].str.contains('entity/Q') == True])/len(df.iloc[:, 0])


0.9996052075650399

In [74]:
df.loc[df.iloc[:, 0].str.contains('entity/Q') == False].iloc[:, 0].str.contains('direct/P')== True

1321       False
9399       False
9751        True
11048       True
12435      False
           ...  
2044386    False
2044815     True
2051081    False
2054171     True
2056536     True
Name: 0, Length: 812, dtype: bool

In [72]:
len(df.loc[df.iloc[:, 1].str.contains('direct/P') == True])/len(df.iloc[:, 1])

0.8314318956308827

In [77]:
len(df.loc[df.iloc[:, 1].str.contains('direct/P') == True])/len(df.loc[df.iloc[:, 1].str.contains('P')])

1.0

1. Check if all the properties are included
   1. if so, we can directly use them. Yes!!!
   ~~2. Otherwise, we can add the ones we don't have~~
2. Use a map to store the properties and the link, ~~both prop/direct/P and entity/P~~
3. Check if all Q has a description
   1. Results: Not, and description is not the name of Q
4. Use url and json to get the name of Q

In [130]:
df_0 = df.loc[df.iloc[:, 0].str.contains('P') == True]
s1 = df_0.iloc[:, 0].unique()

In [139]:
s2 = df.loc[df.iloc[:, 1].str.contains('P')].iloc[:, 1].unique()

In [140]:
len(s2)

251

In [134]:
df.loc[df.iloc[:, 0].str.contains('Q193835') & df.iloc[:, 2].str.contains('Q25186')]
# filem, (director), name, 

,0,1,2
806747,<http://www.wikidata.org/entity/Q193835>,<http://www.wikidata.org/prop/direct/P57>,<http://www.wikidata.org/entity/Q25186>


In [135]:
df.loc[df.iloc[:, 2].str.contains('Q193835') & df.iloc[:, 0].str.contains('Q25186')]
# filem, (director), name, 

,0,1,2


In [93]:
true_count = 0
for i in s2:
    if i in list(s1):
        true_count += 1
        print(df.iloc[list(s1).index(i)])
    else: 
        print(i)

len(s2) - true_count


0     <http://www.wikidata.org/entity/Q1494230>
1    <http://www.wikidata.org/prop/direct/P725>
2     <http://www.wikidata.org/entity/Q3090109>
Name: 295, dtype: object
0     <http://www.wikidata.org/entity/Q2742024>
1    <http://www.wikidata.org/prop/direct/P161>
2      <http://www.wikidata.org/entity/Q866999>
Name: 61, dtype: object
0    <http://www.wikidata.org/entity/Q4040532>
1              <http://schema.org/description>
2                     2012 film by J. T. Petty
Name: 271, dtype: object
0      <http://www.wikidata.org/entity/Q1369749>
1    <http://www.wikidata.org/prop/direct/P1412>
2         <http://www.wikidata.org/entity/Q1860>
Name: 248, dtype: object
0      <http://www.wikidata.org/entity/Q371885>
1    <http://www.wikidata.org/prop/direct/P161>
2      <http://www.wikidata.org/entity/Q193555>
Name: 178, dtype: object
0    <http://www.wikidata.org/entity/Q5115597>
1    <http://www.wikidata.org/prop/direct/P27>
2         <http://www.wikidata.org/entity/Q30>
Name: 208, dtyp

0

In [95]:
df_0 = df.loc[df.iloc[:, 0].str.contains('/Q') == True]
s1 = df_0.iloc[:, 0].unique()
len(s1)

158822

In [108]:
s2 = df.loc[df.iloc[:, 1].str.contains('description')]
len(s2)

150006

In [109]:
s2.iloc[0]

0    <http://www.wikidata.org/entity/Q2358294>
1              <http://schema.org/description>
2                               Canadian actor
Name: 1, dtype: object

In [104]:
df_1 = df.loc[~df.iloc[:, 1].str.contains('Q')]
df_1 = df_1.loc[~df.iloc[:, 1].str.contains('P')]
df_1.iloc[:, 1].unique()

array(['<http://schema.org/description>',
       '<http://www.w3.org/2000/01/rdf-schema#label>',
       '<http://ddis.ch/atai/tag>', '<http://ddis.ch/atai/rating>'],
      dtype=object)

In [129]:
from urllib.request import urlopen
import json
import time
start = time.time()
current_time = start

Q_name_map = dict()
name_Q_map = dict()
for i, url in enumerate(s1):
    Q_entity = str(url).removeprefix('<http://www.wikidata.org/entity/').removesuffix('>')
    response = urlopen(url)
    data_json = json.loads(response.read())
    name = data_json['entities'][Q_entity]['labels']['en']['value']
    Q_name_map.update({Q_entity: name})
    name_Q_map.update({name: Q_entity})
    print('{} / {} {}: {} time: {}/{}'.format(i, len(s1), Q_entity,name, time.time() - current_time, time.time() - start))
    current_time = time.time()


0 / 158822 Q20720659: Kothanodi time: 2.8897640705108643/2.8897669315338135
1 / 158822 Q2358294: Dan Hennessey time: 2.995152235031128/5.885257005691528
2 / 158822 Q30139610: Back to Burgundy time: 1.9233181476593018/7.808861970901489
3 / 158822 Q897357: The Ketchup Effect time: 1.8177189826965332/9.626726865768433
4 / 158822 Q3373174: Simon Norrthon time: 2.092794895172119/11.719619035720825
5 / 158822 Q4071602: Sergey Astakhov time: 2.153406858444214/13.873252868652344
6 / 158822 Q93443: St Trinian's time: 2.434100866317749/16.307619094848633
7 / 158822 Q278053: Labyrinth time: 2.1261708736419678/18.434004068374634
8 / 158822 Q2965738: Christiane Bailly time: 1.9055042266845703/20.33971905708313
9 / 158822 Q55907451: Midsommar time: 2.0495121479034424/22.38942003250122
10 / 158822 Q386389: Paul Brinegar time: 1.9940500259399414/24.38385319709778
11 / 158822 Q104836893: Ajan Kloss time: 1.4659860134124756/25.849998235702515
12 / 158822 Q450646: Edie Adams time: 1.9911670684814453/27.8

KeyboardInterrupt: 